## Step 1. 기계 번역 이야기
---
![image](https://user-images.githubusercontent.com/63500940/93839119-dbd33e80-fcc6-11ea-90d7-79a14a1f8ae7.png)  
  
1940~1980년대까지의 기계번역은 **규칙 기반의 기계 번역(Rule-based Machine Translation)** 이 주를 이루었다.  
  
80년대 이후 **통계적 기계 번역(Statistical machine Translation)**  
  
통계 기반 번역에 여러 머신 러닝이 사용되었지만, 딥 러닝은 아니었다.  
  
구글은 구글 번역기에 **신경망 기게 번역(Neural Machine Translation, NMT)** 을 도입  
  
이때 사용된 인공 신경망이 **seq2seq** 

## 시퀀스 처리하는 RNN
![image](https://user-images.githubusercontent.com/63500940/93839679-adeef980-fcc8-11ea-82ad-0123419409b8.png)  

## seq2seq
### seq2seq의 인코더-디코더 구조
![image](https://user-images.githubusercontent.com/63500940/93839833-235aca00-fcc9-11ea-8c10-8a6cc4e385f5.png)  
  
아래 그림에서 Encoder는 Feature Extractor의 역할을 한다. 어떤 데이터 X를 해석하기 위한 저차원의 feature vector z를 만들어낸다. 반면 Decoder는 저차원의 feature z로부터 정보를 복원해서 다시 어떤 데이터 X'을 재생성하는 역할을 한다.  
![image](https://user-images.githubusercontent.com/63500940/93839892-4f764b00-fcc9-11ea-8613-09d751f2d442.png)  
  
디코더는 인코더의 마지막 time step의 hidden state를 전달 받아 자신의 초기 hidden state로 하고, 출력 문장을 생성해내기 시작한다. 여기서는 특수 문자를 사용해서 출력 문장의 시작과 종료를 알려주어야 한다. 위의 그림에서는 _GO와 EOS가 각각 시작 문자와 종료 문자에 해당한다.  
문헌에 따라서는 SOS와 EOS라고도 하는데, SOS는 start of sequence를 의미하고,  
EOS는 end of sequence를 의미한다.  
  
## Conditional Language Model
문장 생성기(Text Generator) 모델을 만들어 보았다면, 그러한 문장 생성기는 **언어 모델(Language Model)**을 구현한 것이라는 것을 알고 있을 것이다.  
  
언어 모델이란 n-1개의 단어 시퀀스 $w_1, \dots , w_{n-1}$ 가 주어졌을 때, n 번째 단어 $w_n$ 으로 무엇이 올지를 예측하는 확률 모델이다. 파라미터 $\theta$ 로 모델링하는 언어 모델을 다음과 같이 표현할 수 있다.  
$$P(w_n|w_1, \dots ,w_{n-1};\theta)$$  
  
우리가 알고 있는 RNN 계열의 모델들은 이러한 언어 모델을 구현하기에 적합한 것들이다.  
  
그런데, 언어모델에 기반한 문장 생성기가 가지고 있는 한가지 문제점이 있다. 그것은 바로  
[어떤 말을 만들고 싶은지를 제어할 수 없다]는 점이다.RNN 모델이 확률적으로 그 다음 나올 단어들을 순차적으로 만들어 나가게 되는데, 그것을 상황에 맞게 제어할 수 있다면 아주 유용할 것이다. 그래서 사람들은 위 언어모델을 확장해서 조건적 언어모델(Conditional Language MOdle)의 개념을 생각하게 된다. 말하자면 아래와 같다.  
$$P(w_n|w_1, \dots ,w_{n-1},C;\theta)$$  
위 식과 다르게 C라는 것이 추가되었다. c를 이용해 아무 문장이나 만들지 말고 c에 적합한 문장을 만들어라고 주문하고 싶은 것이다.  
  
X라는 영어문장을 Y라는 프랑스어 문장으로 번역해 => 프랑스어 문장 Y를 만들어 봐, 단 그 문장은 영어 X라는 뜻이어야한다.  
  
어떻게 이런 조건을 문장생성기에 넣어줄 수 있을까?  
바로 문장 X를 해석해서 c로 만든 인코더를 또 다른 RNN으로 만드는 것이다.  
그렇게 만든 C를 다시 문장생성기인 디코더 RNN에 입력으로 넣어주는 모델을 만들어 낸 것이 바로 오늘 다루게 될 seq2seq이다.

## 교사 강요(teacher forcing)
seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다르다는 특징이 있다.  
테스트 과정에서는 이전 타입스텝의 출력을 현재 타임스탭의 입력으로 사용한다는 특징을 가지고 있엇지만, 학습 과정에서 만약 이전 스텝에서 생성된 단어가 잘 못된 단어라면 학습을 하기에 굉장히 오랜 시간이 필요할 것이다.  
그래서 교사 강요방식으로 훈련을 하게된다.  
훈련과정에서는 실제 정답을 알고 있기때문에 이전 타임 스텝의 예측값을 현재 타임스텝의 입력으로 사용하는 것이 아니라 이전 타임스텝의 실제값으로 사용할 수 있다.  
학습의 효율은 올라가겟지만 창의적인 문장생성이 어렵다는 단점이 있다.  
![image](https://user-images.githubusercontent.com/63500940/93841604-0b864480-fccf-11ea-866e-186c78b857d1.png)

## 단어 수준 vs 문자 수준
seq2seq는 **단어 수준(world level)** 또는 **문자 수준(character level)** 두 가지 방법으로 구현할 수 있다. 단어 수준이라고 함은 각 RNN의 time step의 입출력 단위가 단어 수준이라는 의미이고, 문자 수준이라 함음 RNN의 time steop의 입출력 단위가 문자 수준. 영어에서는 알파벳 단위이다.  
  
실제 구현 자체는 문자 수준이 좀 더 쉬운데,그 이유는 단어 수준 seq2seq의 경우 매 타임스텝에서 디코더 RNN이 출력을 결정할 때, 훈련 데이터에 있는 전체 단어 집합의 수가 30만개라면 30만개 중 하나를 선택해야 하므로 출력층의 크기가 무려 30만이어야 한다.  
  
하지만 문자 수준으로 구현하면, 영문자 알파벳은 26개에 불과하므로 출력층의 크기가 굉장히 작아진다.  
여기에 대, 소문자를 구분하지 않고 특수문자를 포함한다고 하더라도 출력층의 크기는 100을 넘지 않는다.  
  
그렇다면 단어 수준과 문자 수준의 번역 둘 중 어느 쪽이 유리한가?  
  
두 방법 모두 장단점이 있다. 그리고 그 장단점은 서로 trade-off관계이기도 하다.  
  
'먹다'라는 단어 하나에 많은 변종이 있다. 먹다, 먹는다, 먹고, 먹지 등등. 이 의미가 모두 조금씩 다르다.  
이렇게 따지만 단어 사전의 크기는 거의 무한대에 가까울 것이다.  
이렇듯 단어 수준으로 접근하기엔 사전의 방대함이 걸린다. 또 한국어 일본어 같은 언어에서 띄어쓰기를 많이 신경을 써야하기 때문에 어려움이 있다.  
  
그럼 문자 수준으로 번역하면 어떨까? 문장 -> 단어 -> 문자로 쪼갠다는 것은 문자 수준의 글자 임베딩은 단어 안에 내재된 정보가 손실된다는 것을 의미한다. 즉, 기계가 데이터를 통해 글자가 단어를 이루어 내는 패턴까지 학습을 해내야한다. 그래서 충분한 데이터가 확보되지 않았다면 일반적으로 문자 수준의 번역이 단어 수준의 번역보다 품질이 떨어진다.  
  
하지만 최신 자연어처리의 흐르음은 단어 수준이나 문자 수준의 번역이 아닌, 그 사이의 subword 기반의 번역이 주를 이루고 있다.  

---
이번 시간에는 문자 수준으로 구축을 해보자 그리고 일단 문자 수준 번역기를 구축하고 나면 단어 수준 모델로 변환하는 것은 쉽다

## 번역기 만들기 (1) 데이터 전처리
---
```plaintext
$ mkdir -p ~/aiffel/translator_seq2seq/data
$ mkdir -p ~/aiffel/translator_seq2seq/models
$ wget https://www.manythings.org/anki/fra-eng.zip
$ mv fra-eng.zip  ~/aiffel/translator_seq2seq/data
$ cd ~/aiffel/translator_seq2seq/data && unzip fra-eng.zip
```

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
115056,Do you really think it's no good?,Pensez-vous vraiment que ce ne soit pas bon ?,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
101623,What do you know about pandas?,Que savez-vous des pandas ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
144858,You couldn't have picked a better spot.,Vous n'auriez pas pu choisir meilleur endroit.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
173583,Lincoln's biography is read by children all ar...,La biographie de Lincoln est lue par des enfan...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
23470,Get rid of the gun.,Débarrassez-vous de l'arme !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


세번째 열 제거하고, 훈련 데이터 5만개의 샘플로 줄이기

In [7]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.head(5)

,eng,fra
0,Go.,\t Va ! \n
1,Hi.,\t Salut ! \n
2,Hi.,\t Salut. \n
3,Run!,\t Cours ! \n
4,Run!,\t Courez ! \n


seq2seq 동작을 위해서 디코더의 입력과 예측에는 시작 토큰 <sos>와 종료 토큰 <eos>가 필요하다. 이번에는 각각 `\t` 와 `\n` 을 사용하겠다. 두 토큰을 추가하자

In [5]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
20980,That is his house.,\t C'est sa maison. \n
767,I'm ready.,\t Je suis prêt. \n
14688,I didn't see you.,\t Je ne vous ai pas vue. \n
15983,Mail this letter.,\t Poste cette lettre. \n
34512,Get that book for me.,\t Attrape ce livre pour moi. \n


이제 단어장(vocabulary)을 만들고, 각 단어에 부여된 고유한 정수로 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거친다. 이때 영어와 프랑스어 각각 단어장을 별도로 만들어 준다. 그리고 정상적으로 정수 시퀀스로 변환되었는지 3개의 행을 출력 시켜보자

In [6]:
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 3, 8], [10, 5, 8], [10, 5, 8]]

In [14]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[11, 1, 19, 4, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 14, 1, 12]]

In [15]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73


영어 데이터와 프랑스어 데이터의 최대 길이를 각각 구해보자. 이는 패딩(`<pad>`)을 위함이다.  
모델에 입력될 영어, 프랑스어 시퀀스의 길이가 일정해야 하므로,  
최대 길이로 맞추고 남는 시퀀스 뒷부분을 패딩으로 채우게 된다.

In [16]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 23
프랑스어 시퀀스의 최대 길이 76


In [17]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 23
프랑스어 시퀀스의 최대 길이 76


인코더의 입력으로 사용되는 영어 시퀀스와 달리, 프랑스어 시퀀스는 2가지 버전으로 나누어 준비해야 합니다. 하나는 디코더의 출력과 비교해야 할 정답 데이터로 사용해야 할 원래 목적에 따른 것입니다. 그리고 다른 하나는 이전 스텝에서 언급했던 교사 강요(Teacher forcing)을 위해 디코더의 입력으로 사용하기 위한 것입니다.  
  
이때, 디코더의 입력으로 사용할 시퀀스는 < eos >토큰이 필요가 없고, 디코더의 출력과 비교할 시퀀스는 < sos >가 필요가 없기 때문입니다. 가령, 영어로 'I am a person'이라는 문장을 프랑스어 'Je suis une personne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측하도록 훈련되므로, 이런 방식으로 생성된 두가지 버전의 시퀀스를 준비해야 합니다.

In [18]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [19]:
print(decoder_input[:3])
print(decoder_target[:3])

[[11, 1, 19, 4, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 14, 1]]
[[1, 19, 4, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 14, 1, 12]]


디코더의 입력의 경우에는 숫자 12(< eos > 토큰)가 제거되었고, 디코더의 출력의 경우에는 숫자 11(< sos > 토큰)이 제거되었습니다. 이제 패딩을 진행합니다. 패딩을 진행하면 모든 샘플들의 길이가 정해준 길이로 동일하게 변환됩니다. 여기서는 아까 저장해두었던 가장 긴 샘플의 길이인 **`max_eng_seq_len`, `max_fra_seq_len`** 를 각각 사용한다.  
  
이렇게 되면 영어 데이터의 모든 샘플들은 `max_eng_seq_len`의 길이를 가지고, 프랑스어의 모든 샘플들은 `max_fra_seq_len`의 길이가 될것이다.

In [20]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


모든 샘플들의 길이가 동일하게 변환된 것을 알 수 있다. 모든 샘플들의 길이가 동일하게 변환되는 과정에서 정해준 길이보다 짧은 데이터들은 뒤에 0(<pad>)으로 채워진다.  
인코더의 샘플 하나를 출력해보자

In [21]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


앞서 [19, 3, 8]이라는 3개의 단어만 있단 샘플이 뒤에 0이 채워지면서 `max_eng_seq_len`의 값인 23의 길이를 가지게 되었습니다. 이제 각 정수에 대해서 벡터화 방법으로 원-핫 인코딩을 선택합니다.

In [22]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23, 51)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


원-핫 인코딩을 하고나서의 데이터의 크기는 (샘플의 수 × 샘플의 길이 × 단어장의 크기)가 됩니다. 원-핫 인코딩은 각 정수를 단어장의 크기를 가지는 원-핫 벡터로 인코딩하는 과정이기 때문입니다.

마지막으로, 훈련과정의 validation을 위해 위에서 생성한 데이터 50000건 중 3000건만 검증데이터로 삼고, 나머지를 학습데이터로 삼겠습니다.

In [23]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 23, 51)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


## 번역기 만들기 (2) 모델 훈련하기
이번 실습은 기본적으로 케라스 창시자 프랑수아 숄레의 케라스의 seq2seq 구현 가이드 게시물인  
<li><a href='https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html'> A ten-minute introduction to sequence-to-sequence learning</a></li> 를 참고로 하였습니다.  
  
우선 필요한 도구들을 임포트하자

In [24]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print('⏳')

⏳


먼저 인코더 설계하기  
  
LSTM 셀을 설계하고 문장을 입력받으면 LSTM 셀이 마지막 time step의 hidden state와 cell state를 전달 받아 저장해준다. 앞서 인코더의 마지막 hidden state를 디코더의 첫번째 hidden state로 사용한다고 했었다.  
일반적인 RNN의 경우에는 그것이 맞지만, 기본 RNN보다 좀 더 복잡한 LSTM의 경우 hidden state뿐 아니라, cell state라는 것이 존재한다.  
  
그래서 인코더의 LSTM 셀의 마지막 time step의 hidden state와 cell state를 디코더 LSTM의 첫번째 hidden state와 cell state로 전달해주어야 한다.

In [26]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

>위의 코드를 한 줄, 한 줄 파악해볼게요.
>
>첫번째 줄 : 우선 LSTM의 입력 텐서를 정의해줍니다. 입력 문장을 저장하게 될 변수 텐서입니다.
>
>두번째 줄 : 256의 hidden_size를 가지는 LSTM 셀을 만들어줍니다. LSTM의 수용력(capacity)를 의미합니다. >return_state = True를 해서 hidden state와 cell state를 리턴받을 수 있도록 합니다.
>
>세번째 줄 : 입력 텐서를 입력으로 마지막 time step의 hidden state와 cell state를 결과로 받습니다.
>
>네번째 줄 : 마지막 time step의 hidden state와 cell state를 encoder_states라는 하나의 변수에 저장해뒀습니다. 이를 디코더에 전달하면 되겠네요.
>
>이제 디코더를 설계해볼까요? 디코더도 인코더랑 몇 가지 세부 사항을 제외하고 거의 똑같습니다.

In [27]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

세번째 줄을 보면 디코더의 인자로 `initial_state`가 추가되었는데요. _LSTM 셀의 초기 상태를 정의_ 해줄 수 있는 인자입니다. 여기서는 이전에 저장한 인코더의 `마지막 time step`의 `hidden state`와 `cell state`를 사용하였습니다. 디코더의 출력층을 설계해줍니다.

In [28]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

매 time step마다의 다중 클래스 분류 문제이므로 프랑스어 단어장으로부터 한 가지 문자만 선택하도록 합니다. Dense의 인자로 프랑스어 단어장의 크기를 기재하고, 활성화 함수로 소프트맥스 함수를 사용. 최종적으로 인코더와 디코더를 연결해서 하나의 모델로 만들어줍니다. Model의 Input과 Output의 정의를 유심히 살펴 주세요.

In [32]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

model.fit([encoder_input_train, decoder_input_train], decoder_target_train,
          batch_size=512,
          epochs=10,
          validation_split=0.2)

Epoch 1/10
74/74 [==============================] - 5s 62ms/step - loss: 1.3534 - val_loss: 1.5184
Epoch 2/10
74/74 [==============================] - 4s 53ms/step - loss: 0.9638 - val_loss: 1.0394
Epoch 3/10
74/74 [==============================] - 4s 53ms/step - loss: 0.7880 - val_loss: 0.8843
Epoch 4/10
74/74 [==============================] - 4s 53ms/step - loss: 0.6780 - val_loss: 0.7891
Epoch 5/10
74/74 [==============================] - 4s 53ms/step - loss: 0.6143 - val_loss: 0.7359
Epoch 6/10
74/74 [==============================] - 4s 55ms/step - loss: 0.5747 - val_loss: 0.6839
Epoch 7/10
74/74 [==============================] - 4s 54ms/step - loss: 0.5372 - val_loss: 0.6506
Epoch 8/10
74/74 [==============================] - 4s 55ms/step - loss: 0.5133 - val_loss: 0.6229
Epoch 9/10
74/74 [==============================] - 4s 54ms/step - loss: 0.4878 - val_loss: 0.6103
Epoch 10/10
74/74 [==============================] - 4s 54ms/step - loss: 0.4656 - val_loss: 0.5747


## 번역기 만들기(3) 모델 테스트 하기
seq2seq는 훈련할 때와 테스트 단계의 동작이 다릅니다. 이를 위해서 테스트 단계의 디코더 모델은 설계를 다시 해줄 필요가 있습니다. 물론 이전에 학습된 디코더 모델의 레이어는 그대로 사용합니다. 왜 이렇게 번거로운 과정이 생기는 것일까요? Text Generator 모델을 만들어 보신 분이라면 알 수 있습니다. 훈련시에는 학습해야 할 타겟 문장을 디코더 모델의 입력, 출력 시퀀스로 넣어 주고, 디코더 모델이 타겟 문장을 한꺼번에 출력하게 할 수 있습니다. 그러나 테스트 단계에서는 그럴 수가 없습니다. 하나의 문장을 만들어 내기 위해 루프를 돌면서 단어를 하나씩 차례차례 예측해서, 예측한 단어가 다시 다음 단어를 예측할 때 사용되는 입력으로 재사용되는 과정이 진행되기 때문입니다.  
  
정리하면, 테스트 단계에서의 디코더의 동작 순서는 아래와 같습니다.  
  
1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.
2. 토큰인 `\t`를 디코더에 입력한다.
3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 한다.
4. 3을 반족하다가 토큰인 `\n`이 예측되면 이를 중단한다.  
  
이를 구현하기 위해선 time step의 출력층의 예측 결과를 현재 time step의 입력으로 사용하는 단계를 추가하기 위해서 루프를 돌며 디코더의 LSTM 셀을 마치 수동 제어하는 느낌으로 설계해야 한다.  
코드가 좀 더 길어지게 된다.  
  
우선 인코더를 정의한다. encoder_inputs와 encoder_states는 이미 정의한 것들을 재사용한다.

In [33]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 51)]        0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 315392    
Total params: 315,392
Trainable params: 315,392
Non-trainable params: 0
_________________________________________________________________


디코더 설계하기

In [36]:
#이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
#이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
#이전 time step의 hidden state와 cell state를 하나의 변수에 저장 
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#decoder_states_inputs를 현재 time step의 초기 상태로 사용.
#구체적인 동작 자체는 def decode_sequence()에 구현
decoder_outputs, state_h, state_c, = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
#현재 time_step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

이전 time step의 출력 결과를 현재 time step의 입력으로 사용하기 위해 디코더 LSTM 셀의 동작을 좀 더 세분화해서 구현했다.  
동작 자체는 이후에 구현할 def decode_sequence()에서 좀 더 자세히 다루겠다.  
  
디코더의 출력층을 재설계해준다.

In [37]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs]+decoder_states)
decoder_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  337920      input_3[0][0]                    
                                                                 input_8[0][0]              

단어에서 정수로, 정수에서 단어로 바꾸는 사전을 준비해 둡니다. 테스트 결과를 해석하기 위해선 사전이 필요할 것이다.  
우리는 이전 스텝에서 문장을 숫자 인덱스로 바꾸는 Tokenizer를 만들면서 자동으로 만들어진 사전을 이미 가지고 있다.

In [44]:
eng2ind = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

이제 예측 과정을 위한 함수 `decode_sequence()`를 구현한다. `decode_sequence()`의 입력으로 들어가는 것은 번역하고자 하는 문장의 정수 시퀀스이다. `decode_sequence()` 내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자 하는 문장의 정수 시퀀스인 `input_seq`를 입력하면, encoder_model은 마지막 시점의 hidden state를 리턴한다.  
```plaintext
# decode_sequence() 함수 내에 있는 코드
states_value = encoder_model.predict(input_seq)
```
이 hidden state는 디코더의 첫번째 시점의 hidden state가 되고, 디코더는 이제 번역 문장을 완성하기 위한 예측 과정을 진행한다. 디코더의 예측 과정에서는 이전 시점에서 예측한 단어를 디코더의 현재 시점의 입력으로 넣어주는 작업을 진행한다. 그리고 이 작업은 종료를 의미하는 종료 토큰을 만나거나, 주어진 최대 길이를 넘을 때까지 반복한다.

In [45]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [46]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장:  ne sois pas de pronde ! 
-----------------------------------
입력 문장: I left.
정답 문장:  Je suis partie. 
번역기가 번역한 문장:  je ne suis pas de pronde. 
-----------------------------------
입력 문장: Call us.
정답 문장:  Appelez-nous ! 
번역기가 번역한 문장:  ne sois pas de pronde ! 
-----------------------------------
입력 문장: How nice!
정답 문장:  Comme c'est gentil ! 
번역기가 번역한 문장:  c'est un pronde le maine. 
-----------------------------------
입력 문장: Turn left.
정답 문장:  Tourne à gauche. 
번역기가 번역한 문장:  me ne suis pas de mointe. 
